In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from imblearn.combine import SMOTETomek   # 过采样+欠采样结合的方法处理非平衡数据集

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier as ada
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

#  Voting 
data = pd.read_csv('第一问所有特征.csv',encoding='utf-8',index_col='企业代号')
for i in range(len(data)):
    a='E'+str(i+1)
    if data.loc[a,'是否违约']=='否':
        data.loc[a,'违约']=0
    else :
        data.loc[a,'违约']=1

x = data.iloc[:,:-4].values
y = data.iloc[:,-1].values

In [61]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=30)
smote_tomek = SMOTETomek(random_state=0)
x_train, y_train = smote_tomek.fit_resample(x_train, y_train)

In [62]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(learning_rate=0.05,n_estimators=100,objective='binary',
                           boosting_type='gbdt',
                           num_leaves=2**5,
                           max_depth=5,reg_alpha=0.5,reg_lambda=0.5,
                           metric='auc',subsample=0.75)

LR = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
Ada = ada(algorithm='SAMME', base_estimator=None, learning_rate=0.1,
                   n_estimators=100, random_state=30)
GBDT = GradientBoostingClassifier(ccp_alpha=0.0, learning_rate=0.7,  max_depth=3)
svc = SVC(C=0.8, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=20, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

rf = RF(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0,min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=30, verbose=0,
                       warm_start=False)


sclf = StackingCVClassifier(classifiers=[Ada, GBDT, LR,rf],
                            use_probas=True,
                            meta_classifier=svc,
                            random_state=30)

In [63]:
tranfer = StandardScaler()
x = tranfer.fit_transform(x)
x_train = tranfer.transform(x_train)
x_test = tranfer.transform(x_test)

In [64]:
Ada

AdaBoostClassifier(algorithm='SAMME', learning_rate=0.1, n_estimators=100,
                   random_state=30)

In [65]:
sclf = StackingCVClassifier(classifiers=[lgb, GBDT,rf],
                            use_probas=True,
                            meta_classifier=svc,
                            random_state=30)

zhibiao = {}

# weight = []
for clf, label in zip([LR, Ada, GBDT, svc, rf, lgb,sclf],
                      ['LR',
                       'Ada',
                       'GBDT',
                       'svc',
                       'rf', 'LightGBM','StackingClassifier']):
    clf.fit(x_train, y_train)
    y_predict = clf.predict(x_test)
#     print('{}在预测集模型的准确率为：\n'.format(label), metrics.accuracy_score(y_test, y_predict))
#     print('{}在训练集模型的准确率为：\n'.format(label), metrics.accuracy_score(y_train, clf.predict(x_train)))
#     print('{}的综合准确率为：\n'.format(label), metrics.accuracy_score(y, clf.predict(x)))
#     print('{}的Precision为：'.format(label), precision_score(y_test, y_predict))
#     print('{}的Recall为：'.format(label), recall_score(y_test, y_predict))
    tem = metrics.roc_auc_score(y_test, y_predict)
#     weight.append(tem)
#     print('{}的ROC面积为：'.format(label), metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]))
#     print('{}的f1值为：'.format(label), metrics.f1_score(y_test, y_predict))
#     print()
    
    tem_1 = [metrics.accuracy_score(y_train, clf.predict(x_train)),metrics.accuracy_score(y_test, y_predict),
            precision_score(y_test, y_predict),recall_score(y_test, y_predict),
            metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]),metrics.f1_score(y_test, y_predict)]
    zhibiao[label]=tem_1
data2 = pd.DataFrame(data=zhibiao,index=['train_Accuracy','test_Accuracy',
                                'Precision','Recall','AUC','f1']).T
data2

,train_Accuracy,test_Accuracy,Precision,Recall,AUC,f1
LR,0.867188,0.838710,0.400000,0.50,0.759259,0.444444
Ada,0.953125,0.741935,0.333333,1.00,0.847222,0.500000
GBDT,1.000000,0.806452,0.375000,0.75,0.907407,0.500000
svc,1.000000,0.903226,1.000000,0.25,0.500000,0.400000
rf,1.000000,0.838710,0.444444,1.00,0.953704,0.615385
LightGBM,0.984375,0.774194,0.333333,0.75,0.916667,0.461538
StackingClassifier,0.984375,0.967742,0.800000,1.00,0.981481,0.888889


In [57]:
# LGB单模(对非平衡数据集设置is_unbalance)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=30)
clf = LGBMClassifier(learning_rate=0.05,n_estimators=100,objective='binary',
                           boosting_type='gbdt',
                           num_leaves=2**5,
                           max_depth=5,reg_alpha=0.5,reg_lambda=0.5,is_unbalance=True,
                           metric='auc',subsample=0.75)
clf.fit(x_train,y_train)
pd.DataFrame(data=[metrics.accuracy_score(y_train, clf.predict(x_train)),metrics.accuracy_score(y_test, y_predict),
            precision_score(y_test, y_predict),recall_score(y_test, y_predict),
            metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]),metrics.f1_score(y_test, y_predict)],index=['train_Accuracy','test_Accuracy',
                                'Precision','Recall','AUC','f1'],columns=['LightGBM']).T

,train_Accuracy,test_Accuracy,Precision,Recall,AUC,f1
LightGBM,0.956522,0.967742,0.8,1.0,0.805556,0.888889
